# Scenariusze testowe dla porównania wydajności baz danych

### 1. Operacja CREATE

- Dodanie nowego nauczyciela
- Utworzenie nowej klasy
- Dodanie nowego przedmiotu
- Zarejestrowanie nowego ucznia
- Przypisanie ucznia do klasy
- Utworzenie harmonogramu zajęć
- Wystawienie oceny

### 2. Operacja READ

Pobranie kompleksowego raportu zawierającego:
- Dane osobowe ucznia
- Informacje o klasie
- Dane nauczyciela prowadzącego
- Listę ocen z opisami przedmiotów
- Szczegółowy harmonogram zajęć

### 3. Operacja UPDATE

- Aktualizacja danych ucznia
- Zmiana przypisania do klasy
- Modyfikacja nazwy klasy
- Aktualizacja danych nauczyciela
- Zmiana oceny
- Aktualizacja opisu przedmiotu
- Modyfikacja harmonogramu zajęć

### 4. Operacja DELETE

- Usunięcie ocen ucznia
- Wypisanie ucznia z klasy
- Usunięcie harmonogramu zajęć
- Usunięcie klasy
- Opcjonalne usunięcie przedmiotów
- Opcjonalne usunięcie nauczyciela
- Usunięcie rekordu ucznia

## Ilość rekordów do testów

Testy będą przeprowadzane dla następujących ilości rekordów:

1. 10,000 rekordów
2. 100,000 rekordów
3. 1,000,000 rekordów
4. 10,000,000 rekordów

## Metryki wydajnościowe

Dla każdego scenariusza i ilości rekordów będziemy mierzyć:

1. Czas wykonania całego scenariusza
2. Średni czas pojedynczych operacji
3. Liczbę operacji na sekundę (throughput)
4. Zużycie zasobów systemowych (CPU, RAM, I/O dysku)

# Narzędzia i technologie testowe

### Wbudowane instrumenty bazodanowe

Każdy system oferuje specjalizowane narzędzia diagnostyczne:


| System | Narzędzie | Funkcjonalności |
| :-- | :-- | :-- |
| PostgreSQL | pgBench | Testy TPC-B, własne skrypty SQL |
| MariaDB | sysbench | Testy OLTP, skalowanie pionowe |
| MongoDB | mongoperf | Operacje na dokumentach JSON |
| Cassandra | cassandra-stress | Testy dystrybucji danych |
| Redis | redis-benchmark | Pomiar opóźnień operacji klucz-wartość |

Wykorzystanie natywnych narzędzi pozwala na precyzyjne badanie specyficznych mechanizmów storage engine.

### Automatyzacja w Pythonie

Kluczowe biblioteki wspierające testy:

- **SQLAlchemy** dla baz relacyjnych
- **PyMongo** dla MongoDB
- **Cassandra-driver** dla Cassandra
- **redis-py** dla Redis


In [1]:
import psycopg2
from pymongo import MongoClient
from cassandra.cluster import Cluster
import redis
import mysql.connector
import yaml

with open('docker-compose.yml', 'r') as file:
    docker_config = yaml.safe_load(file)

# PostgreSQL connection
postgres_config = docker_config['services']['postgresql']
postgres_client = psycopg2.connect(
    host='localhost',  # or 'postgresql' if running inside Docker network
    database=postgres_config['environment']['POSTGRES_DB'],
    user=postgres_config['environment']['POSTGRES_USER'],
    password=postgres_config['environment']['POSTGRES_PASSWORD'],
    port=postgres_config['ports'][0].split(':')[0]
)

# MariaDB connection
mariadb_config = docker_config['services']['mariadb']
mariadb_client = mysql.connector.connect(
    host='localhost',  # or 'mariadb' if running inside Docker network
    database=mariadb_config['environment']['MYSQL_DATABASE'],
    user=mariadb_config['environment']['MYSQL_USER'],
    password=mariadb_config['environment']['MYSQL_PASSWORD'],
    port=mariadb_config['ports'][0].split(':')[0]
)

# Cassandra connection
cassandra_config = docker_config['services']['cassandra']
cassandra_client = Cluster(['localhost'],  # or ['cassandra'] if running inside Docker network
                        port=cassandra_config['ports'][0].split(':')[0])
cassandra_session = cassandra_client.connect()

# MongoDB connection
mongo_config = docker_config['services']['mongodb']
mongo_client = MongoClient(
    host='localhost',  # or 'mongodb' if running inside Docker network
    port=int(mongo_config['ports'][0].split(':')[0])
)

# Redis connection
redis_config = docker_config['services']['redis']
redis_client = redis.Redis(
    host='localhost',  # or 'redis' if running inside Docker network
    port=int(redis_config['ports'][0].split(':')[0])
)

In [2]:
# Simple sanity checks for each database connection

# PostgreSQL check
try:
    with postgres_client.cursor() as cursor:
        cursor.execute("SELECT 1")
        print("PostgreSQL connection successful")
except Exception as e:
    print(f"PostgreSQL connection failed: {e}")

# MariaDB check
try:
    with mariadb_client.cursor(buffered=True) as cursor:
        cursor.execute("SELECT 1")
        print("MariaDB connection successful")
except Exception as e:
    print(f"MariaDB connection failed: {e}")


# Cassandra check
try:
    cassandra_session.execute("SELECT release_version FROM system.local")
    print("Cassandra connection successful")
except Exception as e:
    print(f"Cassandra connection failed: {e}")

# MongoDB check
try:
    mongo_client.admin.command('ping')
    print("MongoDB connection successful")
except Exception as e:
    print(f"MongoDB connection failed: {e}")

# Redis check
try:
    redis_client.ping()
    print("Redis connection successful")
except Exception as e:
    print(f"Redis connection failed: {e}")


PostgreSQL connection successful
MariaDB connection successful
Cassandra connection successful
MongoDB connection successful
Redis connection successful
